In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.downloader

In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import files
uploaded = files.upload()

import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
train

Saving train.csv to train (2).csv


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
from google.colab import files
uploaded = files.upload()

test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
test

Saving test.csv to test (2).csv


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [5]:
##Importing Libraries for Neural Nets
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, GlobalAvgPool1D, GlobalMaxPool1D
import math
from tqdm import tqdm
import pickle 
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from tqdm.keras import TqdmCallback

In [6]:
# download glove twitter embeddings
pprint(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']


In [27]:
# Takes about 5 minutes to execute, for 100-dim twitter vectors
# Takes about 10+ minutes to execute, for 200-dim twitter vectors
# glove_vectors_100 = gensim.downloader.load('glove-twitter-100')
glove_vectors_200 = gensim.downloader.load('glove-twitter-200')
glove = glove_vectors_200
embedding_length = 200


## https://nlp.stanford.edu/projects/glove/

In [28]:
print(train.shape)
print(test.shape)

text_train = np.array(train['text'])
text_test = np.array(test['text'])
text = np.concatenate((text_train, text_test), axis = 0)

(7613, 5)
(3263, 4)


In [29]:
text

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ..., 'Green Line derailment in Chicago http://t.co/UtbXLcBIuY',
       'MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3',
       '#CityofCalgary has activated its Municipal Emergency Plan. #yycstorm'],
      dtype=object)

In [30]:
# word tokenization 
for i in range(len(text)):
    text[i] = word_tokenize(text[i])
print('After tokenization:')
print(text[0])

# filter out punctuation
for i in range(len(text)):
    text[i] = [word for word in text[i] if word.isalpha()]
print('After filtering out punctuation:')
print(text[0])

# make words lowercase 
for i in range(len(text)):
    text[i] = [word.lower() for word in text[i]]
print('After making lowercase:')
print(text[0])

# remove stopwords
for i in range(len(text)):
    text[i] = [word for word in text[i] if not word in stop_words]
print('After removing stopwords:')
print(text[0])

# concatenate list of words
for i in range(len(text)):
    text_concat = ''
    for word in text[i]:
        text_concat += word + ' '
    text[i] = text_concat
print('After concatenating words:')
text = np.array(text)

After tokenization:
['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', '#', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all']
After filtering out punctuation:
['Our', 'Deeds', 'are', 'the', 'Reason', 'of', 'this', 'earthquake', 'May', 'ALLAH', 'Forgive', 'us', 'all']
After making lowercase:
['our', 'deeds', 'are', 'the', 'reason', 'of', 'this', 'earthquake', 'may', 'allah', 'forgive', 'us', 'all']
After removing stopwords:
['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us']
After concatenating words:


In [31]:
glove = glove_vectors_200 

In [32]:
# glove.most_similar('apple')

In [33]:
print(glove.get_vector('twitter').shape)
glove.get_vector('twitter')

(200,)


array([ 1.0433e-01,  3.9882e-01, -6.9542e-01, -5.0984e-01,  6.6959e-02,
       -6.6277e-01,  2.5302e-01, -6.1320e-01,  6.6954e-02,  3.4897e-01,
       -3.4739e-01, -4.3558e-01,  3.0712e-01, -4.9802e-01, -9.2228e-01,
       -4.3034e-01,  8.4605e-02, -2.7696e-01,  3.6586e-01, -5.0237e-01,
        1.4885e-02,  2.1380e-01, -2.1657e-01, -6.2373e-02, -4.4156e-01,
       -1.6118e+00,  2.9537e-01, -1.0533e+00, -1.0280e+00,  9.4658e-01,
       -2.4770e-01,  1.2826e-01,  1.6348e-01, -4.6791e-01,  6.4240e-01,
       -1.6270e-01,  5.0066e-01, -1.4123e-01, -3.0530e-01, -2.2070e-01,
       -4.9756e-01,  4.8674e-01, -1.9606e-01,  4.7974e-01,  2.0614e-01,
       -6.0490e-02, -2.7333e-01, -2.8277e-01, -2.5695e-01,  1.5094e-01,
        5.7955e-01, -5.3770e-01,  1.7460e-02, -3.0005e-01, -8.4298e-02,
        3.6802e-01, -1.0117e+00,  2.3048e-01, -5.8210e-02, -2.3774e-01,
        6.5609e-03, -2.8949e-01, -2.7410e-01,  2.0116e-01, -1.4284e-01,
        2.1115e-01,  8.4995e-02,  3.3736e-01,  2.8317e-01,  3.37

In [40]:
embeddings = []
for sentence in text:
    embedding = np.zeros(200)
    word_count = 0
    for word in sentence:
        if word in glove.vocab:
            embedding += glove.get_vector(word)
        word_count += 1
    if word_count != 0:
        embedding /= word_count
    embeddings.append(embedding)
embeddings = np.array(embeddings)

In [41]:
# embeddings

In [42]:
X_train = embeddings[:7613]
X_test = embeddings[7613:] 

y_train = train['target'].values

In [43]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(7613, 200)
(3263, 200)
(7613,)


In [44]:
embeddings = []
for sentence in text:
    embedding = []
    for word in sentence.split():
        if word in glove.vocab:
            embedding.append(glove.get_vector(word).tolist())
        else:
            embedding.append(np.zeros(embedding_length).tolist())
    # pad extra zeros to make length of each sentence = 22 
    while len(embedding) < 22:
        embedding.append(np.zeros(embedding_length).tolist())
    embeddings.append(embedding)
embeddings = np.array(embeddings)
print(embeddings.shape)

(10876, 22, 200)


In [45]:
embeddings

array([[[-0.31435001,  0.27825001, -0.12037   , ...,  0.17955001,
         -0.75700003, -1.29920006],
        [ 0.13054   ,  0.045089  , -0.058965  , ...,  0.68779999,
          0.25959   , -0.24171001],
        [-0.32275   ,  0.14165001,  0.053551  , ...,  0.064692  ,
          0.53280997,  0.41641   ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[-0.48640001,  0.07589   ,  0.11724   , ..., -1.10109997,
         -1.00610006,  0.42403001],
        [-0.48187   , -0.29707   , -0.77723002, ..., -0.96424001,
          0.065379  ,  0.20376   ],
        [-0.43726   , -0.16597   ,  0.32686001, ..., -0.43044999,
          0.45885   ,  0.21791001],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  

In [46]:
X_train = embeddings[:7613]
X_test = embeddings[7613:]
y_train = train['target'].values

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(7613, 22, 200)
(3263, 22, 200)
(7613,)


In [47]:
act = 'tanh'
batch_len = 32
opt = 'adam'
epoch = 20
val_split = 0.2

In [53]:
keras.backend.clear_session()
model = Sequential()
model.add(Input(shape = (22, embedding_length)))
model.add(LSTM(22))
model.add(Dense(32, activation = act))
# model.add(Dense(32, activation = act))
model.add(Dense(1, activation = 'sigmoid')) 
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [49]:
model.fit(X_train, y_train, epochs = epoch, batch_size = batch_len, verbose = 1, validation_split=val_split)

Epoch 1/20
191/191 [==============================] - 6s 19ms/step - loss: 0.5112 - accuracy: 0.7617 - val_loss: 0.4433 - val_accuracy: 0.8076
Epoch 2/20
191/191 [==============================] - 3s 14ms/step - loss: 0.4248 - accuracy: 0.8199 - val_loss: 0.4325 - val_accuracy: 0.8162
Epoch 3/20
191/191 [==============================] - 3s 14ms/step - loss: 0.3998 - accuracy: 0.8350 - val_loss: 0.4358 - val_accuracy: 0.8181
Epoch 4/20
191/191 [==============================] - 3s 13ms/step - loss: 0.3827 - accuracy: 0.8415 - val_loss: 0.4240 - val_accuracy: 0.8181
Epoch 5/20
191/191 [==============================] - 3s 15ms/step - loss: 0.3624 - accuracy: 0.8539 - val_loss: 0.4328 - val_accuracy: 0.8267
Epoch 6/20
191/191 [==============================] - 3s 14ms/step - loss: 0.3430 - accuracy: 0.8642 - val_loss: 0.4424 - val_accuracy: 0.8142
Epoch 7/20
191/191 [==============================] - 2s 13ms/step - loss: 0.3180 - accuracy: 0.8764 - val_loss: 0.4551 - val_accuracy: 0.8122

In [50]:
model.predict(X_test)

array([[0.997677  ],
       [0.31636244],
       [0.9983692 ],
       ...,
       [0.9990643 ],
       [0.8418813 ],
       [0.9989203 ]], dtype=float32)

In [39]:
# y_pred = model.predict(X_test)
# y_pred = []
# for i in range(len(y_test)):
#     if y_train[i] > 0.5:
#         y_train_pred.append(1)
#     else:
#         y_train_pred.append(0)
# # test['target'] = y_pred
# # final = test[['id', 'target']]
# # final.to_csv('pred.csv', index = False)
# # final

In [35]:
from sklearn import metrics